<a href="https://colab.research.google.com/github/sid0312/ANPR/blob/master/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
import cv2
import glob,os,time
from PIL import Image

In [0]:
json_path = '/content/drive/My Drive/Indian_Number_plates.json'

In [0]:
data = pd.read_json(json_path,lines=True)

In [26]:
data.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN


In [0]:
os.mkdir('License Plates')

In [7]:
data['annotation'][0]

[{'imageHeight': 466,
  'imageWidth': 806,
  'label': ['number_plate'],
  'notes': '',
  'points': [{'x': 0.722084367245657, 'y': 0.5879828326180251},
   {'x': 0.8684863523573201, 'y': 0.688841201716738}]}]

In [8]:
print(len(data))

237


In [9]:
dataset = {}
vals = ['name','height','width','top_x','top_y','bottom_x','bottom_y']
for i in vals:
  dataset[i] = list()
count = 0
for idx,row in data.iterrows():
  img = urllib.request.urlopen(row["content"])
  img = Image.open(img)
  img = img.convert('RGB')
  img.save("/content/License Plates/car_{}.jpeg".format(count),'JPEG')
  dataset['name'].append('car_{}'.format(count))
  annotation = row['annotation']
  dataset['height'].append(annotation[0]['imageHeight'])
  dataset['width'].append(annotation[0]['imageWidth'])
  dataset['top_x'].append(annotation[0]['points'][0]['x'])
  dataset['top_y'].append(annotation[0]['points'][0]['y'])
  dataset['bottom_x'].append(annotation[0]['points'][1]['x'])
  dataset['bottom_y'].append(annotation[0]['points'][1]['y']) 
  count =count+1
  if count%10==0:
    print('Downloaded.. {}/{}'.format(count,len(data)))  
  if count%len(data)==0:
    print('Loading last image.. {}/{}'.format(count,len(data)))  
print('{} images downloaded'.format(count))                                              

Downloaded.. 10/237
Downloaded.. 20/237
Downloaded.. 30/237
Downloaded.. 40/237
Downloaded.. 50/237
Downloaded.. 60/237
Downloaded.. 70/237
Downloaded.. 80/237
Downloaded.. 90/237
Downloaded.. 100/237
Downloaded.. 110/237
Downloaded.. 120/237
Downloaded.. 130/237
Downloaded.. 140/237
Downloaded.. 150/237
Downloaded.. 160/237
Downloaded.. 170/237
Downloaded.. 180/237
Downloaded.. 190/237
Downloaded.. 200/237
Downloaded.. 210/237
Downloaded.. 220/237
Downloaded.. 230/237
Loading last image.. 237/237
237 images downloaded


In [0]:
df = pd.DataFrame(dataset)

In [28]:
df.head()

,name,height,width,top_x,top_y,bottom_x,bottom_y
0,car_0,466,806,0.722084,0.587983,0.868486,0.688841
1,car_1,449,494,0.161943,0.850780,0.582996,1.000000
2,car_2,314,500,0.082000,0.697452,0.230000,0.828025
3,car_3,469,500,0.434000,0.667377,0.718000,0.765458
4,car_4,563,1001,0.208791,0.390764,0.865135,0.984014


In [0]:
df.to_csv('license_plates.csv',index=False)

In [0]:
csv_path = '/content/license_plates.csv'

In [49]:
licenses = pd.read_csv(csv_path)
licenses.tail()

,name,height,width,top_x,top_y,bottom_x,bottom_y
232,car_232,366,500,0.344000,0.609290,0.706000,0.778689
233,car_233,333,500,0.368000,0.453453,0.616000,0.618619
234,car_234,332,500,0.334000,0.608434,0.628000,0.768072
235,car_235,331,586,0.295222,0.601208,0.441980,0.719033
236,car_236,147,395,0.040506,0.129252,0.929114,0.857143


In [0]:
licenses['name'] = licenses['name'] + '.jpeg'

In [51]:
licenses.head()

,name,height,width,top_x,top_y,bottom_x,bottom_y
0,car_0.jpeg,466,806,0.722084,0.587983,0.868486,0.688841
1,car_1.jpeg,449,494,0.161943,0.850780,0.582996,1.000000
2,car_2.jpeg,314,500,0.082000,0.697452,0.230000,0.828025
3,car_3.jpeg,469,500,0.434000,0.667377,0.718000,0.765458
4,car_4.jpeg,563,1001,0.208791,0.390764,0.865135,0.984014


In [0]:
licenses.drop(columns=['height','width'],inplace=True)

In [53]:
licenses.head()

,name,top_x,top_y,bottom_x,bottom_y
0,car_0.jpeg,0.722084,0.587983,0.868486,0.688841
1,car_1.jpeg,0.161943,0.850780,0.582996,1.000000
2,car_2.jpeg,0.082000,0.697452,0.230000,0.828025
3,car_3.jpeg,0.434000,0.667377,0.718000,0.765458
4,car_4.jpeg,0.208791,0.390764,0.865135,0.984014


In [0]:
widths,heights =[],[]
center_x,center_y=[],[]
for i in range(len(licenses)):
  widths.append(licenses['bottom_x'][i]- licenses['top_x'][i])
  heights.append(licenses['bottom_y'][i]- licenses['top_y'][i])
  center_x.append((licenses['bottom_x'][i] + licenses['top_x'][i])/2)
  center_y.append((licenses['bottom_y'][i] + licenses['top_y'][i])/2)
lic = pd.DataFrame(columns=['x','y','w','h'])
lic['w'] = widths
lic['h'] = heights
lic['x'] = center_x
lic['y'] = center_y

In [55]:
lic.head()

,x,y,w,h
0,0.795285,0.638412,0.146402,0.100858
1,0.372470,0.925390,0.421053,0.149220
2,0.156000,0.762739,0.148000,0.130573
3,0.576000,0.716418,0.284000,0.098081
4,0.536963,0.687389,0.656344,0.593250


In [0]:
for i in range(len(lic)):
  f = open('/content/License Plates/car_{}.txt'.format(i),'w+')
  f.write('{} {} {} {} {}'.format(0,lic['x'][i],lic['y'][i],lic['w'][i],lic['h'][i]))
  f.close()

In [63]:
!zip -r /content/file.zip /content/License\ Plates
from google.colab import files
files.download("/content/file.zip")

  adding: content/License Plates/ (stored 0%)
  adding: content/License Plates/car_86.jpeg (deflated 1%)
  adding: content/License Plates/car_83.txt (deflated 27%)
  adding: content/License Plates/car_64.txt (deflated 28%)
  adding: content/License Plates/car_154.jpeg (deflated 1%)
  adding: content/License Plates/car_21.jpeg (deflated 1%)
  adding: content/License Plates/car_209.jpeg (deflated 1%)
  adding: content/License Plates/car_217.txt (deflated 37%)
  adding: content/License Plates/car_190.jpeg (deflated 2%)
  adding: content/License Plates/car_91.jpeg (deflated 1%)
  adding: content/License Plates/car_23.jpeg (deflated 0%)
  adding: content/License Plates/car_102.txt (deflated 29%)
  adding: content/License Plates/car_228.jpeg (deflated 1%)
  adding: content/License Plates/car_231.jpeg (deflated 2%)
  adding: content/License Plates/car_46.txt (deflated 27%)
  adding: content/License Plates/car_199.jpeg (deflated 0%)
  adding: content/License Plates/car_163.jpeg (deflated 1%)
 

# **Extracted the file in my local system and uploaded it in my google drive as well as github repository**